In [ ]:
import pandas  as pd

def read_data(fname, num_sent=4):
    contexts = []
    endings = []
    labels = []
    data = pd.read_csv(fname)
    for idx, row in data.iterrows():
        sents = []
        for i in [4,3,2,1]:
            if len(sents) == num_sent:
                break
            sents.insert(0, row[f'Kalimat-{i}'])
        context = ' '.join(sents) # row['Kalimat-1'] +' '+ row['Kalimat-2'] +' '+ row['Kalimat-3'] +' '+ row['Kalimat-4']
        ending1 = row['Correct Ending']
        ending2 = row['Incorrect Ending']
        
        contexts.append(context)
        endings.append(ending1)
        labels.append(1)
        
        contexts.append(context)
        endings.append(ending2)
        labels.append(0)
    return contexts, endings, labels

In [ ]:
import pyrouge, shutil
import logging, os

pyrouge_tools = '/home/ffajri/Workspace/pyrouge/tools/ROUGE-1.5.5'
def rouge_eval(ref_dir, dec_dir):
    """Evaluate the files in ref_dir and dec_dir with pyrouge, returning results_dict"""
    r = pyrouge.Rouge155(pyrouge_tools)
    r.model_filename_pattern = '#ID#'
    r.system_filename_pattern = '(\d+)'
    r.model_dir = ref_dir
    r.system_dir = dec_dir
    logging.getLogger('global').setLevel(
        logging.WARNING)  # silence pyrouge logging
    rouge_results = r.convert_and_evaluate()
    return r.output_to_dict(rouge_results)


def rouge(context, option):
    gold_temp = 'temp_gold'
    pred_temp = 'temp_pred'
        
    if os.path.exists(gold_temp):
        shutil.rmtree(gold_temp)
        shutil.rmtree(pred_temp)
    os.makedirs(gold_temp)
    os.makedirs(pred_temp)
    open(gold_temp+'/0', 'w').write(context)
    open(pred_temp+'/0', 'w').write(option)
    
    results_dict = rouge_eval(gold_temp, pred_temp)
    return results_dict['rouge_1_f_score']

In [ ]:
for num_sent in [1,2,3,4]:
    scores = []; true = 0; idy = 0
    contexts, endings, _ = read_data('Data/data_id/test.csv', num_sent)
    for idx, context in enumerate(contexts):
        scores.append(rouge(context, endings[idx]))
    while idy < len(scores):
        if scores[idy] > scores[idy+1]:
            true+=1        
        idy+=2
    print(num_sent, true/(len(contexts)/2))